In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 41 kB/s 
     |████████████████████████████████| 199 kB 62.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=00d08bcb06ab998ff93b88263e6263d76ba2acc9471e57e4512378aae93ca0e1
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [62]:
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.functions import col,lit,split,bround

In [4]:
spark= SparkSession.builder.appName("test").getOrCreate()

# New Section

In [14]:
df2 = spark.read.option('header',True).csv('/content/County_Population.csv',inferSchema=True)

In [17]:
df2.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Id2: integer (nullable = true)
 |-- County: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Population Estimate 2018: integer (nullable = true)



In [28]:
df2.count()

3220

In [55]:
df2.dropDuplicates(['Id2'])

DataFrame[Id: string, Id2: int, County: string, State: string, Population Estimate 2018: int]

In [56]:
df2.count()

3220

In [57]:
df2.drop('na','null')

DataFrame[Id: string, Id2: int, County: string, State: string, Population Estimate 2018: int]

In [58]:
df2.count()

3220

In [33]:
df2.show(5)

+--------------+----+-------+-------+------------------------+
|            Id| Id2| County|  State|Population Estimate 2018|
+--------------+----+-------+-------+------------------------+
|0500000US01001|1001|Autauga|Alabama|                   55601|
|0500000US01003|1003|Baldwin|Alabama|                  218022|
|0500000US01005|1005|Barbour|Alabama|                   24881|
|0500000US01007|1007|   Bibb|Alabama|                   22400|
|0500000US01009|1009| Blount|Alabama|                   57840|
+--------------+----+-------+-------+------------------------+
only showing top 5 rows



In [35]:
df2.groupby('State').count().show(5)

+---------+-----+
|    State|count|
+---------+-----+
|     Utah|   29|
|   Hawaii|    5|
|Minnesota|   87|
|     Ohio|   88|
| Arkansas|   75|
+---------+-----+
only showing top 5 rows



In [36]:
df2.groupby('State').max().show(5)

+---------+--------+-----------------------------+
|    State|max(Id2)|max(Population Estimate 2018)|
+---------+--------+-----------------------------+
|     Utah|   49057|                      1152633|
|   Hawaii|   15009|                       980080|
|Minnesota|   27173|                      1259428|
|     Ohio|   39175|                      1310300|
| Arkansas|    5149|                       392680|
+---------+--------+-----------------------------+
only showing top 5 rows



In [37]:
df2.groupby('county').min().show(5)

+---------+--------+-----------------------------+
|   county|min(Id2)|min(Population Estimate 2018)|
+---------+--------+-----------------------------+
|Worcester|   24047|                        51823|
|   Aitkin|   27001|                        15902|
|Petroleum|   30069|                          513|
| Thurston|   31173|                         7303|
|    Bucks|   42017|                       628195|
+---------+--------+-----------------------------+
only showing top 5 rows



In [38]:
df2.groupby('State').avg().show(5)

+---------+--------+-----------------------------+
|    State|avg(Id2)|avg(Population Estimate 2018)|
+---------+--------+-----------------------------+
|     Utah| 49029.0|           109003.62068965517|
|   Hawaii| 15005.0|                     284098.2|
|Minnesota| 27087.0|            64496.31034482759|
|     Ohio| 39088.0|           132834.56818181818|
| Arkansas|  5075.0|           40184.333333333336|
+---------+--------+-----------------------------+
only showing top 5 rows



In [41]:
df2.groupby('State').sum().show(4)

+---------+--------+-----------------------------+
|    State|sum(Id2)|sum(Population Estimate 2018)|
+---------+--------+-----------------------------+
|     Utah| 1421841|                      3161105|
|   Hawaii|   75025|                      1420491|
|Minnesota| 2356569|                      5611179|
|     Ohio| 3439744|                     11689442|
+---------+--------+-----------------------------+
only showing top 4 rows



In [43]:
df3 = df2.select('Id2','State','County','Population Estimate 2018')

In [44]:
df3.show(10)

+----+-------+--------+------------------------+
| Id2|  State|  County|Population Estimate 2018|
+----+-------+--------+------------------------+
|1001|Alabama| Autauga|                   55601|
|1003|Alabama| Baldwin|                  218022|
|1005|Alabama| Barbour|                   24881|
|1007|Alabama|    Bibb|                   22400|
|1009|Alabama|  Blount|                   57840|
|1011|Alabama| Bullock|                   10138|
|1013|Alabama|  Butler|                   19680|
|1015|Alabama| Calhoun|                  114277|
|1017|Alabama|Chambers|                   33615|
|1019|Alabama|Cherokee|                   26032|
+----+-------+--------+------------------------+
only showing top 10 rows



In [53]:
df3.count()

3220

In [69]:
df3.groupby('State','County').min('Population Estimate 2018').show()

+--------------+--------------------+-----------------------------+
|         State|              County|min(Population Estimate 2018)|
+--------------+--------------------+-----------------------------+
|       Alabama|             Choctaw|                        12841|
|    California|              Orange|                      3185968|
|       Florida|              Monroe|                        75027|
|       Georgia|             Laurens|                        47325|
| New Hampshire|            Sullivan|                        43144|
|    New Jersey|            Atlantic|                       265429|
|North Carolina|            Davidson|                       166614|
|          Ohio|              Geauga|                        94031|
|        Oregon|           Clackamas|                       416075|
|South Carolina|             Laurens|                        66994|
|     Tennessee|           Claiborne|                        31756|
|         Texas|              Newton|           

In [74]:
df3.agg({'Population Estimate 2018':'sum'}).show()

+-----------------------------+
|sum(Population Estimate 2018)|
+-----------------------------+
|                    330362587|
+-----------------------------+



In [76]:
df3.agg({'Population Estimate 2018':'min'}).show()

+-----------------------------+
|min(Population Estimate 2018)|
+-----------------------------+
|                           88|
+-----------------------------+



In [77]:
df3.agg({'Population Estimate 2018':'max'}).show()

+-----------------------------+
|max(Population Estimate 2018)|
+-----------------------------+
|                     10105518|
+-----------------------------+



In [78]:
df3.agg({'Population Estimate 2018':"avg"}).show()

+-----------------------------+
|avg(Population Estimate 2018)|
+-----------------------------+
|           102597.07670807453|
+-----------------------------+

